In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from IPython.display import display
import pandas as pd

from islets.Recording import Recording#, saveMovie
# from islets.Regions import Regions
# from islets.numeric import power_spectrum, rebin

from matplotlib.colors import LogNorm

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
from process_series import parse_leica

In [4]:
import parsing_1 as parser

In [5]:
parser.update_protocols()

In [6]:
protocols = parser.import_protocols(parseMetadata=False)

deleted  sex as not useful
deleted  frequency as not useful
when pH is NAN I will assume it is neutral (7.4).


In [7]:
protocols["recording"] = [x.split("_")[0] for x in protocols.experiment]
protocols["spreadsheet name"] = ["_".join(x.split("_")[1:]) for x in protocols.experiment]
protocols["Date"] = pd.to_datetime(protocols["date"], format="%Y_%m_%d", errors="coerce")

In [8]:
newProtocols = []

for (date,recname), df in protocols.groupby(["date","recording"]):
    if "Experiment54" not in recname: continue
    found = False
    for ext in ["lif","nd2"]:
        recfile = f"/data/Sandra/{date.split('_')[0]}/{date}/{recname}.{ext}"
        if os.path.isfile(recfile):
            found=True
            break
    if not found:
        print (f"{recname} is not where it is supposed to be ", f"/data/Sandra/{date.split('_')[0]}/{date}/")
        continue
    break
    newProtocols += [df.copy()]
    
    recmeta = f"/data/Sandra/{date.split('_')[0]}/{date}/.{recname}.{ext}.meta"

In [9]:
rec = Recording(recfile)
rec.calc_gaps()
indices = parse_leica(rec, index=True)

for ii, outname in indices:
    if len(ii)>1:
        row = rec.metadata.loc[ii[0]].copy()
        row["Name"] = outname
        row["End time"] = rec.metadata.loc[ii[-1],"End time"]
        row["compound"] = rec.metadata.loc[ii,["Name","Start time","End time","Duration","gap"]].copy()
        rec.metadata.drop(index=ii, inplace=True)
        rec.metadata = rec.metadata.append(row)
rec.metadata["Duration"] = rec.metadata["End time"] - rec.metadata["Start time"]
rec.metadata.sort_values("Name", inplace=True)

merged = pd.merge(rec.metadata,df,
         how="outer", right_on="spreadsheet name",left_on="Name")

# merged["series"] = merged["Name"].copy()

merged["Date"] = df.Date.iloc[0]
merged["recording"] = df.recording.iloc[0]

for k in ["date","experiment"]:
    del merged[k]

In [10]:
firstOrder = ["Date","recording","spreadsheet name","Name","Slice number"]
firstOrder += ['Frequency', 'Duration','SizeT', 'SizeX', 'SizeY', 'pxSize', 'pH', 'glucose', 'treatment', 'comments', 'microscope', 'dye',]
merged = merged[firstOrder+[c for c in merged.columns if c not in firstOrder]]

In [11]:
merged

,Date,recording,spreadsheet name,Name,Slice number,Frequency,Duration,SizeT,SizeX,SizeY,pxSize,pH,glucose,treatment,comments,microscope,dye,pxUnit,bit depth,Start time,End time,gap,mouse/human,part of pancreas,pH_,glucose_,magnification
0,2020-06-18,Experiment54a,Series003,Series003,1.0,9.89,00:12:26.807124,7385,256,256,0.857,7.4,15.0,nan,"31yr old female, BMI = 20.3, HbA1c = 4.8. Prep...",leica CF_SP8,Calbryte-520,µm,uint8,2020-06-18 13:43:48,2020-06-18 13:56:14.807124239,NaN,human,isolated islets,7.4,15,20x
1,2020-06-18,Experiment54a,Series004,Series004,2.0,19.80,00:23:42.698572,28132,128,128,1.530,7.4,8.0,nan,very long events,leica CF_SP8,Calbryte-520,µm,uint8,2020-06-18 14:09:14,2020-06-18 14:32:56.698572968,779.192876,BL6J,NaN,7.4,8,20x


In [44]:
newProtocols = pd.concat(newProtocols,ignore_index=True)

In [45]:
newProtocols

,Date,recording,series,Slice number,date,experiment,microscope,mouse/human,pH,glucose,treatment,comments,dye,part of pancreas,pH_,glucose_,magnification
0,2019-06-10,Experiment21,Series021,1.0,2019_06_10,Experiment21_Series021,Leica upright,NMRI,7.4,12.0,pH changes (7.4 63-1240; 7.1 1240-1856; 7.7 18...,NaN,Calbryte-520,NaN,NaN,12,20x
1,2019-06-10,Experiment21,Series029,2.0,2019_06_10,Experiment21_Series029,Leica upright,NMRI,7.4,8.0,pH changes (7.4 42-1286; 7.1 1286-1920; 7.7 19...,NaN,Calbryte-520,NaN,NaN,8,20x
2,2019-06-10,Experiment21,Series034,3.0,2019_06_10,Experiment21_Series034,Leica upright,NMRI,7.4,8.0,pH changes (7.4 36-1286; 7.1 1286-1921; 7.7 19...,NaN,Calbryte-520,NaN,NaN,8,20x
3,2019-08-13,Experiment29,Series052-060,NaN,2019_08_13,Experiment29_Series052-060,Leica upright,NMRI,7.4,6.0,100 nM ryanodine,NaN,Calbryte-520,NaN,7.4,6,20x
4,2019-08-13,Experiment29,Series061,NaN,2019_08_13,Experiment29_Series061,Leica upright,NMRI,7.4,6.0,100 nM ryanodine,NaN,Calbryte-520,NaN,7.4,6,20x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,2020-07-01,Experiment58c,Series002,3.0,2020_07_01,Experiment58c_Series002,leica CF_SP8,BL6J,7.4,8.0,nan,two islets,Calbryte-520,NaN,7.4,8,20x
341,2020-07-01,Experiment58d,Series001,4.0,2020_07_01,Experiment58d_Series001,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x
342,2020-07-01,Experiment58d,Series003,5.0,2020_07_01,Experiment58d_Series003,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x
343,2020-07-01,Experiment58d,Series006,6.0,2020_07_01,Experiment58d_Series006,leica CF_SP8,BL6J,7.4,8.0,nan,NaN,Calbryte-520,NaN,7.4,8,20x


In [72]:
newProtocols = []
for cur,ds,fs in os.walk("/data/Sandra/"):
    for f in fs:
        if "protocol" not in f: continue
        protocolFilename = os.path.join(cur,f)
        try:
            protocol = pd.read_csv(protocolFilename)
        except:
            continue
#         print(protocolFilename)
#         display(protocol)
        protocol.sort_values(["compound","begin"], inplace=True)
        row = OrderedDict([(cmp,df.concentration.values) for cmp,df in protocol.groupby("compound")])
        # print (f"{protocolFilename} does not contain entry for glucose. #justsaying")
        row["treatment"] = []
        for cmp in protocol["compound"].unique():
            if cmp in [ "glucose"]: #"pH",
                row[cmp] = [float(el.lower().rstrip("m")) for el in row[cmp]]
                for fn in min, max:
                    row[cmp+"_"+fn.__name__] = fn(row[cmp])
            else:
                row["treatment"] += [cmp]
                del row[cmp]
        exp,ser = protocolFilename.split(os.path.sep)[-3:-1]
        exp = exp.split("_analysis")[0]        
        row["Experiment"] = exp
        row["Series"] = ser
        row["location"] = os.path.split(protocolFilename)[0]
        newProtocols += [row]

In [76]:
newProtocols = pd.DataFrame(newProtocols)
firstCols = ["Experiment","Series","glucose",]+["pH"]*int("pH" in newProtocols)+["treatment","location"]
newProtocols = newProtocols[firstCols + [c for c in newProtocols.columns if c not in firstCols]]
newProtocols.sort_values(firstCols[:2])

,Experiment,Series,glucose,treatment,location,glucose_min,glucose_max
3,Experiment59a.lif,Series003-4_0-5160s,[8.0],[pH],/data/Sandra/2020/2020_07_08/Experiment59a.lif...,8.0,8.0
4,Experiment59b.lif,Series002-5,"[6.0, 8.0]",[pH],/data/Sandra/2020/2020_07_08/Experiment59b.lif...,6.0,8.0
2,Experiment59c.lif,Series002-5,"[6.0, 8.0, 6.0, 8.0, 6.0, 8.0]",[Ca],/data/Sandra/2020/2020_07_08/Experiment59c.lif...,6.0,8.0
1,Experiment59d.lif,Series002-3,"[6.0, 8.0]",[],/data/Sandra/2020/2020_07_08/Experiment59d.lif...,6.0,8.0
0,Experiment59d.lif,Series005-7,"[8.0, 6.0, 6.0]",[ACh],/data/Sandra/2020/2020_07_08/Experiment59d.lif...,6.0,8.0
5,Experiment60a.lif,Series007,[8.0],[],/data/Sandra/2020/2020_07_15/Experiment60a.lif...,8.0,8.0
6,Experiment60b.lif,Series007-10,"[6.0, 8.0]",[pH],/data/Sandra/2020/2020_07_15/Experiment60b.lif...,6.0,8.0
7,Series003.nd2,all,[8.0],[pH],/data/Sandra/2020/2020_06_22/Series003.nd2_ana...,8.0,8.0


In [51]:
row

OrderedDict([('glucose', [8.0]),
             ('pH', [7.4, 7.1]),
             ('treatment', []),
             ('glucose_min', 8.0),
             ('glucose_max', 8.0),
             ('pH_min', 7.1),
             ('pH_max', 7.4)])

In [52]:
protocol

,compound,concentration,begin,end
0,glucose,8mM,NaN,NaN
1,pH,7.4,NaN,3:00
2,pH,7.1,5:00,NaN


In [34]:
min.__name__

'min'

In [23]:
from collections import OrderedDict

In [32]:
tmp

OrderedDict([('glucose', [8.0]), ('pH', array([7.4, 7.1]))])

In [6]:
javabridge.kill_vm()